# Mozilla Treeherder API

Import dependencies

In [10]:
import requests
from pprint import pprint
import datetime
from thclient import TreeherderClient
import pandas as pd
from datetime import datetime
import numpy as np
from collections import Counter

Initialize the Treeherder Client

In [11]:
client = TreeherderClient()

Get a single job

In [4]:
job_params = {
    "id":492088703
}

client.get_jobs("mozilla-central", **job_params)

[{'build_architecture': '-',
  'build_os': '-',
  'build_platform': 'linux1804-64-shippable-qr',
  'build_platform_id': 667,
  'build_system_type': 'taskcluster',
  'end_timestamp': 1738049633,
  'failure_classification_id': 1,
  'id': 492088703,
  'job_group_description': '',
  'job_group_id': 991,
  'job_group_name': 'Talos performance tests',
  'job_group_symbol': 'T',
  'job_guid': '11a208d8-f2e6-47ed-950e-d77215196a72/0',
  'job_type_description': '',
  'job_type_id': 316474,
  'job_type_name': 'test-linux1804-64-shippable-qr/opt-talos-svgr',
  'job_type_symbol': 's',
  'last_modified': '2025-01-28T07:33:54.080857',
  'machine_name': 't-linux64-ms-007',
  'machine_platform_architecture': '-',
  'machine_platform_os': '-',
  'option_collection_hash': '102210fe594ee9b33d82058545b1ed14f4c8206e',
  'platform': 'linux1804-64-shippable-qr',
  'push_id': 1569927,
  'reason': 'scheduled',
  'ref_data_name': 'd07415df104f84fd9fb6eb187f0f055ecb639d1d',
  'result': 'success',
  'result_set_i

Get performance summaries for a specific period and specific infrastructure

In [69]:
performance_summary_params = {
    "repository": "mozilla-central",
    "signature":308858,
    "interval":2592000,
    "all_data":True,
    "replicates":False
}

data_list = client._get_json("performance/summary", **performance_summary_params)
data_dict = data_list[0]
for key in data_dict:
    print(key, " -> ", data_dict[key])
    if key == "data":
        jobs_list = data_dict[key]

signature_id  ->  308858
framework_id  ->  1
signature_hash  ->  0985b500d96a14de78703763a0851390602daded
platform  ->  linux1804-64-shippable-qr
test  ->  
suite  ->  tscrollx_paint_skip
lower_is_better  ->  True
has_subtests  ->  True
tags  ->  
values  ->  []
name  ->  tscrollx_paint_skip opt e10s fission stylo webrender
parent_signature  ->  None
job_ids  ->  []
repository_name  ->  mozilla-central
repository_id  ->  1
data  ->  [{'job_id': 492088703, 'id': 1945748730, 'value': 0.25810430543303653, 'push_timestamp': '2025-01-28T04:37:39', 'push_id': 1569927, 'revision': '70cabc8df49e79271982f4484a15a2a259d5159b'}, {'job_id': 492123306, 'id': 1945921123, 'value': 0.2600211802824455, 'push_timestamp': '2025-01-28T09:01:11', 'push_id': 1570040, 'revision': 'fb62e31afd93c99bbbffc4528c4773ba46b0fa4b'}, {'job_id': 492191563, 'id': 1946271254, 'value': 0.256399779500452, 'push_timestamp': '2025-01-28T16:18:37', 'push_id': 1570420, 'revision': 'd9b57e2b8726c289eb9ba86c2bc81594bbfbd326'}, {

Get all performance test frameworks

In [ ]:
client._get_json("performance/framework")

Get performance test framework by id

In [ ]:
performance_framework_id = 1
performance_framework_endpoint = f"performance/framework/{performance_framework_id}"

client._get_json(performance_framework_endpoint)

Get all repositories

In [ ]:
repos_list = client._get_json("repository")
for repo in repos_list:
  print(repo['name'])

Get all machine platforms

In [ ]:
client._get_json("machineplatforms")

Get all machine platforms for a particular branch/project that has performance test data

In [ ]:
client._get_json("performance/platforms", "autoland")

Get performance test signatures for a given project

In [5]:
signature_summary_params = {
    # "framework":1,
    # "platform":"linux1804-64-shippable-qr",
    "id": 308858 # mozilla central
    # "id": 307933 # same test for autoland
}

data_dict = client._get_json("performance/signatures", "mozilla-central", **signature_summary_params)
for key in data_dict:
    print(key, " -> ", data_dict[key])

308858  ->  {'id': 308858, 'signature_hash': '0985b500d96a14de78703763a0851390602daded', 'framework_id': 1, 'option_collection_hash': '102210fe594ee9b33d82058545b1ed14f4c8206e', 'machine_platform': 'linux1804-64-shippable-qr', 'suite': 'tscrollx_paint_skip', 'should_alert': True, 'application': 'firefox', 'has_subtests': True, 'extra_options': ['e10s', 'fission', 'stylo', 'webrender']}


Get all the jobs/performance test times for a certain signature for the last 30 days for mozilla central

In [ ]:
performance_summary_params = {
    "repository": "mozilla-central",
    "signature":308858,
    "interval":2592000,
    "all_data":True,
    "replicates":False
}

data_list = client._get_json("performance/summary", **performance_summary_params)
data_dict = data_list[0]
jobs_list = data_dict['data']
for index, job in enumerate(jobs_list):
  job_params = {
      "id":job["job_id"]
  }
  single_job_list = client.get_jobs("mozilla-central", **job_params)
  job_dict = single_job_list[0]
  submit_time = datetime.fromtimestamp(job_dict['submit_timestamp'])
  start_time = datetime.fromtimestamp(job_dict['start_timestamp'])
  end_time = datetime.fromtimestamp(job_dict['end_timestamp'])
  duration = end_time - start_time
  wait_time = end_time - submit_time

  if index == 0:
    prev_submit_time = submit_time

  submit_time_diff = submit_time - prev_submit_time
  prev_submit_time = submit_time


  print("id: ", job_dict['id'],
        " -> ",
        "   submit time: ", submit_time,
        "   start time: ", start_time,
        "   end time: ", end_time,
        "   duration: ", duration,
        "   wait time: ", wait_time,
        "   submit time difference: ", submit_time_diff)


Get all the jobs/performance test times for a certain signature for the last 30 days for autoland

In [49]:
performance_summary_params = {
    "repository": "autoland",
    "signature":307933,
    "interval":2592000,
    "all_data":True,
    "replicates":False
}

data_list = client._get_json("performance/summary", **performance_summary_params)
data_dict = data_list[0]
jobs_list = data_dict['data']
for index, job in enumerate(jobs_list):
  job_params = {
      "id":job["job_id"]
  }
  single_job_list = client.get_jobs("autoland", **job_params)
  job_dict = single_job_list[0]
  submit_time = datetime.fromtimestamp(job_dict['submit_timestamp'])
  start_time = datetime.fromtimestamp(job_dict['start_timestamp'])
  end_time = datetime.fromtimestamp(job_dict['end_timestamp'])
  duration = end_time - start_time
  wait_time = end_time - submit_time

  if index == 0:
    prev_submit_time = submit_time

  submit_time_diff = submit_time - prev_submit_time
  prev_submit_time = submit_time


  print("id: ", job_dict['id'],
        " -> ",
        "   submit time: ", submit_time,
        "   start time: ", start_time,
        "   end time: ", end_time,
        "   duration: ", duration,
        "   wait time: ", wait_time,
        "   submit time difference: ", submit_time_diff)

id:  491863496  ->     submit time:  2025-01-26 01:28:16    start time:  2025-01-26 03:29:53    end time:  2025-01-26 03:52:37    duration:  0:22:44    wait time:  2:24:21    submit time difference:  0:00:00
id:  491890494  ->     submit time:  2025-01-26 07:38:14    start time:  2025-01-26 09:59:50    end time:  2025-01-26 10:22:15    duration:  0:22:25    wait time:  2:44:01    submit time difference:  6:09:58
id:  491908319  ->     submit time:  2025-01-26 19:52:52    start time:  2025-01-26 21:51:44    end time:  2025-01-26 22:14:30    duration:  0:22:46    wait time:  2:21:38    submit time difference:  12:14:38
id:  491914763  ->     submit time:  2025-01-27 01:35:46    start time:  2025-01-27 03:21:10    end time:  2025-01-27 03:43:57    duration:  0:22:47    wait time:  2:08:11    submit time difference:  5:42:54
id:  491958481  ->     submit time:  2025-01-27 06:05:56    start time:  2025-01-27 08:38:34    end time:  2025-01-27 09:00:58    duration:  0:22:24    wait time:  2:5

Get all performance alerts

In [ ]:
client._get_json("performance/alert")

Extract all the alert summaries for a specific siganture

In [ ]:
alert_summary_params = {
    "alerts__series_signature": 5095204, # autoland
    "timerange": 31536000 # last year
}

client._get_json("performance/alertsummary", **alert_summary_params)

{'count': 6,
 'next': None,
 'previous': None,
 'results': [{'id': 42728,
   'push_id': 1534497,
   'prev_push_id': 1534492,
   'original_revision': '4b1d6ebad4b653f9ba2065ce90fd760ebeac7dc8',
   'created': '2024-11-19T16:19:08.964030',
   'first_triaged': '2024-11-19T16:19:09.309955',
   'triage_due_date': '2024-11-22T16:19:08.964030',
   'repository': 'autoland',
   'framework': 1,
   'alerts': [{'id': 196107,
     'status': 4,
     'series_signature': {'id': 5095204,
      'framework_id': 1,
      'signature_hash': 'f52eec8b7ff1f524d98696c023e50fd3f3d3e790',
      'machine_platform': 'linux1804-64-shippable-qr',
      'suite': 'pdfpaint',
      'test': 'issue1721.pdf',
      'lower_is_better': True,
      'has_subtests': False,
      'option_collection_hash': '102210fe594ee9b33d82058545b1ed14f4c8206e',
      'tags': [''],
      'extra_options': ['e10s', 'fission', 'stylo', 'webrender'],
      'measurement_unit': 'ms',
      'suite_public_name': None,
      'test_public_name': None},

Extract all the alert summaries for a specific siganture.
Extract all the performance summaries, i.e. performance tests/jobs for a specific signature.
Merge the two data frames on push_id so that we can determine if a test has lead to a performance alert

In [8]:
# ### Creating jobs' dataframe

performance_summary_params = {
    "repository": "autoland",
    "signature":5095204,
    "interval":31536000,
    "all_data":True,
    "replicates":False
}

performance_summaries_list = client._get_json("performance/summary", **performance_summary_params)
performance_summaries_dict = performance_summaries_list[0]
jobs_list = performance_summaries_dict['data']

job_push_ids_list = []
job_ids_list = []

for index, job in enumerate(jobs_list):
  if job["job_id"]:
    job_push_ids_list.append(job["push_id"])
    job_ids_list.append(job["job_id"])

jobs_df = pd.DataFrame({'job_id': job_ids_list, 'job_push_id': job_push_ids_list})

### Creating alerts' dataframe

alert_summary_params = {
    "alerts__series_signature": 5095204, # autoland
    "timerange": 31536000 # last year
}

alert_summaries_response_dict = client._get_json("performance/alertsummary", **alert_summary_params)
alert_summaries_list = alert_summaries_response_dict["results"]

alert_ids_list = []
alert_push_ids_list = []
alert_bug_ids_list = []

for alert_summary in alert_summaries_list:
  for alert in alert_summary["alerts"]:    
    alert_ids_list.append(alert["id"])
    alert_push_ids_list.append(alert_summary["push_id"])
    alert_bug_ids_list.append(alert_summary.get("bug_number"))
    

bug_ids_list_without_none = [id for id in alert_bug_ids_list if id is not None]

alerts_df = pd.DataFrame({'alert_id': alert_ids_list, 'alert_push_id': alert_push_ids_list, 'alert_bug_id': alert_bug_ids_list})

pprint(alerts_df)
pprint(bug_ids_list_without_none)
    
# Merge on alert_push_id and job_push_id
jobs_perf_regression_info_df = pd.merge(jobs_df, alerts_df, left_on='job_push_id', right_on='alert_push_id', how='left')

# Drop duplicate job_push_id column
jobs_perf_regression_info_df.drop(columns=['alert_push_id'], inplace=True)

pprint(jobs_perf_regression_info_df)


for index, row in jobs_perf_regression_info_df.iterrows():
  job_params = {
      "id":row["job_id"].astype(int)
  }

  single_job_list = client.get_jobs("autoland", **job_params)

  job_dict = single_job_list[0]
  submit_time = datetime.fromtimestamp(job_dict['submit_timestamp'])
  start_time = datetime.fromtimestamp(job_dict['start_timestamp'])
  end_time = datetime.fromtimestamp(job_dict['end_timestamp'])
  duration = end_time - start_time
  wait_time = end_time - submit_time

  if index == 0:
    prev_submit_time = submit_time

  submit_time_diff = submit_time - prev_submit_time
  prev_submit_time = submit_time


  print("id: ", job_dict['id'],
        " -> ",
        "   submit time: ", submit_time,
        "   wait time: ", wait_time,
        "   alert id: ", row['alert_id'],
        "   perf bug id: ", row['alert_bug_id'])


     alert_id  alert_push_id  alert_bug_id
0      196107        1534497     1931487.0
1      196560        1534497     1931487.0
2      196100        1534533           NaN
3      196101        1534533           NaN
4      196102        1534533           NaN
..        ...            ...           ...
517      3438        1446029           NaN
518      3471        1446029           NaN
519      3423        1446029           NaN
520      3453        1446029           NaN
521      3445        1446029           NaN

[522 rows x 3 columns]
[1931487, 1931487, 1931880, 1931880, 1931880, 1931880, 1931880, 1931880]
        job_id  job_push_id  alert_id  alert_bug_id
0    483760411      1537173       NaN           NaN
1    483790536      1537240       NaN           NaN
2    483827344      1537341       NaN           NaN
3    483857384      1537435       NaN           NaN
4    483876432      1537497       NaN           NaN
..         ...          ...       ...           ...
677  500379613      160

Categorize alerts based on their status

In [12]:
alert_summary_params = {
    "alerts__series_signature": 5095204, # autoland
    "timerange": 31536000 # last year
}

alert_summaries_response_dict = client._get_json("performance/alertsummary", **alert_summary_params)
alert_summaries_list = alert_summaries_response_dict["results"]

alert_ids_list = []
alert_status_list = []

for alert_summary in alert_summaries_list:
  for alert in alert_summary["alerts"]:    
    alert_ids_list.append(alert["id"])
    alert_status_list.append(alert["status"])

status_counts = Counter(alert_status_list)

pprint(status_counts)

Counter({2: 495, 1: 15, 4: 8, 3: 4})
